<a href="https://colab.research.google.com/github/rahiakela/mathematics-for-machine-learning/blob/main/deep-learning-book-maths/2_12_principal_components_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Principal Components Analysis

Dimensions are a crucial topic in data science. The dimensions are all the features of the dataset. For instance, if you are looking at a dataset containing pieces of music, dimensions could be the genre, the length of the piece, the number of instruments, the presence of a singer etc. You can imagine all these dimensions as different columns. When there is only two dimensions, it is very convenient to plot: you can use the $x$- and $y$-axis. Add color and you can represent a third dimension. It is similar if you have tens or hundereds of dimensions, it will just be harder to visualize it.

When you have that many dimensions it happens that some of them are correlated. For instance, we can reasonably think that the genre dimension will correlate with the instruments dimensions in our previous example. One way to reduce dimensionality is simply to keep only some of them. The problem is that you loose good information. It would be nice to have a way to reduce these dimensions while keeping all the information present in the data set.

The aim of principal components analysis (PCA) is generaly to reduce the number of dimensions of a dataset where dimensions are not completly decorelated. PCA provides us with a new set of dimensions, the principal components (PC). They are ordered: the first PC is the dimension having the largest variance. In addition, each PC is orthogonal to the preceding one. Remember that orthogonal vectors means that their dot product is equal to $0$ (see [2.6](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.6-Special-Kinds-of-Matrices-and-Vectors/)). This means that each PC is decorelated to the preceding one. It is way better than feature selection where you loose a lot of information.

##Setup

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Plot style
sns.set()
%pylab inline
pylab.rcParams['figure.figsize'] = (4, 4)

Populating the interactive namespace from numpy and matplotlib


In [ ]:
def plot_vectors(vecs, cols, alpha=1):
  """
  Plot set of vectors.

  Parameters
  ----------
  vecs : array-like
      Coordinates of the vectors to plot. Each vectors is in an array. For
      instance: [[1, 3], [2, 2]] can be used to plot 2 vectors.
  cols : array-like
      Colors of the vectors. For instance: ['red', 'blue'] will display the
      first vector in red and the second in blue.
  alpha : float
      Opacity of vectors

  Returns:

  fig : instance of matplotlib.figure.Figure
      The figure of the vectors
  """
  plt.axvline(x=0, color='#A9A9A9', zorder=0)
  plt.axhline(y=0, color='#A9A9A9', zorder=0)

  for i in range(len(vecs)):
      if (isinstance(alpha, list)):
          alpha_i = alpha[i]
      else:
          alpha_i = alpha
      x = np.concatenate([[0,0],vecs[i]])
      plt.quiver([x[0]],
                  [x[1]],
                  [x[2]],
                  [x[3]],
                  angles='xy', scale_units='xy', scale=1, color=cols[i],
                alpha=alpha_i)

## Describing the problem

###Example 1: Orthogonal vector

Unit vectors are an example of orthogonal vectors:

<img src="https://github.com/rahiakela/mathematics-for-machine-learning/blob/main/deep-learning-book-maths/images/orthogonal-vectors.png?raw=1" width="200" alt="Example of orthogonal vectors" title="Orthogonal vectors">

<em>Orthogonal vectors</em>

## Describing the problem

The problem can be expressed as finding a function that converts a set of data points from $\mathbb{R}^n$ to $\mathbb{R}^l$. This means that we change the number of dimensions of our dataset. We also need a function that can decode back from the transformed dataset to the initial one:

<img src="https://github.com/rahiakela/mathematics-for-machine-learning/blob/main/deep-learning-book-maths/images/principal-components-analysis-PCA-change-coordinates.png?raw=1" width="80%" alt="Principal components analysis (PCA)" title="Principal components analysis (PCA)">

<em>Principal components analysis as a change of coordinate system</em>

The first step is to understand the shape of the data. $x^{(i)}$ is one data point containing $n$ dimensions. 

Let's have $m$ data points organized as column vectors (one column per point):

$$
{x}=
\begin{bmatrix}
    x^{(1)} & x^{(2)} & \cdots & x^{(m)}
\end{bmatrix}
$$

If we deploy the $n$ dimensions of our data points we will have:

$$
{x}=
\begin{bmatrix}
    x_1^{(1)} & x_1^{(2)} & \cdots & x_1^{(m)}\\\\
    x_2^{(1)} & x_2^{(2)} & \cdots & x_2^{(m)}\\\\
    \cdots & \cdots & \cdots & \cdots\\\\
    x_n^{(1)} & x_n^{(2)} & \cdots & x_n^{(m)}
\end{bmatrix}
$$

We can also write:

$$
{x}=
\begin{bmatrix}
    x_1\\\\
    x_2\\\\
    \cdots\\\\
    x_n
\end{bmatrix}
$$

$c$ will have the shape:

$$
{c}=
\begin{bmatrix}
    c_1\\\\
    c_2\\\\
    \cdots\\\\
    c_l
\end{bmatrix}
$$

## Adding some constraints: the decoding function

The encoding function $f({x})$ transforms ${x}$ into ${c}$ and the decoding function transforms back ${c}$ into an approximation of ${x}$. 

To keep things simple, PCA will respect some constraints:

### Constraint 1

The decoding function has to be a simple matrix multiplication:

$$
g({c})={Dc}
$$

By applying the matrix ${D}$ to the dataset from the new coordinates system we should get back to the initial coordinate system.

### Constraint 2

The columns of ${D}$ must be orthogonal (see [2.6](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.6-Special-Kinds-of-Matrices-and-Vectors/)).

### Constraint 3

The columns of ${D}$ must have unit norm (see [2.6](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.6-Special-Kinds-of-Matrices-and-Vectors/)).

## Finding the encoding function

Important: For now we will consider only **one data point**. Thus we will have the following dimensions for these matrices (note that ${x}$ and ${c}$ are column vectors):

<img src="https://github.com/rahiakela/mathematics-for-machine-learning/blob/main/deep-learning-book-maths/images/principal-components-analysis-PCA-decoding-function.png?raw=1" width="250" alt="Principal components analysis (PCA) - the decoding function" title="The decoding function">
<em>The decoding function</em>

We want a decoding function which is a simple matrix multiplication. For that reason, we have $g({c})={Dc}$. We will then find the encoding function from the decoding function. We want to minimize the error between the decoded data point and the actual data point. With our previous notation, this means reducing the distance between ${x}$ and $g({c})$. As an indicator of this distance, we will use the squared $L^2$ norm (see [2.5](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.5-Norms/)):

$$
||{{x} - g({c})}||_2^2
$$

This is what we want to minimize. Let's call ${c}^*$ the optimal ${c}$. Mathematically it can be written:

$$
{c}^* = \underset{c}{\arg\min} ||{{x} - g({c})}||_2^2
$$

This means that we want to find the values of the vector ${c}$ such that $||{{x} - g({c})}||_2^2$ is as small as possible.

If you have a look back to [2.5](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.5-Norms/) you can see that the squared $L^2$ norm can be expressed as:

$$
||{{y}}||_2^2 = {y}^\text{T}{y}
$$

We have named the variable ${y}$ to avoid confusion with our ${x}$. Here ${y}={x} - g({c})$


Thus the equation that we want to minimize becomes:

$$
({x} - g({c}))^\text{T}({x} - g({c}))
$$

Since the transpose respects addition we have:

$$
({x}^\text{T} - g({c})^\text{T})({x} - g({c}))
$$

By the distributive property (see [2.2](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.2-Multiplying-Matrices-and-Vectors/)) we can develop:

$$
{x^\text{T}x} - {x}^\text{T}g({c}) -  g({c})^\text{T}{x} + g({c})^\text{T}g({c})
$$

The commutative property (see [2.2](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.2-Multiplying-Matrices-and-Vectors/)) tells us that $
{x^\text{T}y} = {y^\text{T}x}
$. We can use that in the previous equation: we have $
g({c})^\text{T}{x} = {x}^\text{T}g({c})
$. So the equation becomes:

$$
{x^\text{T}x} -{x}^\text{T}g({c}) -{x}^\text{T}g({c}) + g({c})^\text{T}g({c})\\\\
= {x^\text{T}x} -2{x}^\text{T}g({c}) + g({c})^\text{T}g({c})
$$

The first term ${x^\text{T}x}$ does not depends on ${c}$ and since we want to minimize the function according to ${c}$ we can just get off this term. We simplify to:

$$
{c}^* = \underset{c}{\arg\min} -2{x}^\text{T}g({c}) + g({c})^\text{T}g({c})
$$

Since $g({c})={Dc}$:

$$
{c}^* = \underset{c}{\arg\min} -2{x}^\text{T}{Dc} + ({Dc})^\text{T}{Dc}
$$

With $({Dc})^\text{T}={c}^\text{T}{D}^\text{T}$ (see [2.2](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.2-Multiplying-Matrices-and-Vectors/)), we have:

$$
{c}^* = \underset{c}{\arg\min} -2{x}^\text{T}{Dc} + {c}^\text{T}{D}^\text{T}{Dc}
$$

As we saw in [2.6](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.6-Special-Kinds-of-Matrices-and-Vectors/), ${D}^\text{T}{D}={I}_l$ because ${D}$ is orthogonal (actually, it is [semi-orthogonal](https://en.wikipedia.org/wiki/Semi-orthogonal_matrix) if $n \neq l$) and their columns have unit norm. We can replace in the equation:

$$
{c}^* = \underset{c}{\arg\min} -2{x}^\text{T}{Dc} + {c}^\text{T}{I}_l{c}
$$

$$
{c}^* = \underset{c}{\arg\min} -2{x}^\text{T}{Dc} + {c}^\text{T}{c}
$$

### Minimizing the function

So far so good! Now the goal is to find the minimum of the function $- 2{x}^\text{T}{Dc} + {c}^\text{T}{c}$. One widely used way of doing that is to use the **gradient descent** algorithm. It is not the focus of this chapter but we will say a word about it (see [4.3](http://www.deeplearningbook.org/contents/numerical.html) of the Deep Learning Book for more details). The main idea is that the sign of the derivative of the function at a specific value of $x$ tells you if you need to increase or decrease $x$ to reach the minimum. When the slope is near $0$, the minimum should have been reached.

<img src="https://github.com/rahiakela/mathematics-for-machine-learning/blob/main/deep-learning-book-maths/images/gradient-descent.png?raw=1" width="400" alt="Mechanism of the gradient descent algorithm" title="Mechanism of the gradient descent algorithm">
<em>Gradient descent</em>

However, functions with local minima can trouble the descent:

<img src="https://github.com/rahiakela/mathematics-for-machine-learning/blob/main/deep-learning-book-maths/images/gradient-descent-local-minima.png?raw=1" width="400" alt="Gradient descent in the case of local minimum" title="Gradient descent">
<em>Gradient descent can get stuck in local minima</em>

These examples are in 2 dimensions but the principle stands for higher dimensional functions. The gradient is a vector containing the partial derivatives of all dimensions. Its mathematical notation is $\nabla_xf({x})$.

### Calculating the gradient of the function

Here we want to minimize through each dimension of ${c}$. We are looking for a slope of $0$. The equation is:

$$
\nabla_c(-2{x}^\text{T}{Dc} + {c}^\text{T}{c})=0
$$

Let's take these terms separately to calculate the derivative according to ${c}$. 

$$
\frac{d(-2{x}^\text{T}{Dc})}{d{c}} = -2{x}^\text{T}{D}
$$

The second term is ${c}^\text{T}{c}$. We can develop the vector ${c}$ and calculate the derivative for each element:

$$
\begin{align*}
\frac{d({c}^\text{T}{c})}{d{c}} &=
\left(\frac{d({c}_1^2 + {c}_2^2 + \cdots + {c}_l^2)}{d{c}_1},
\frac{d({c}_1^2 + {c}_2^2 + \cdots + {c}_l^2)}{d{c}_2},
\cdots,
\frac{d({c}_1^2 + {c}_2^2 + \cdots + {c}_l^2)}{d{c}_l}\right) \\\\
&=(2{c}_1, 2{c}_2, \cdots, 2{c}_l)\\\\
&=2({c}_1, {c}_2, \cdots, {c}_l)\\\\
&=2{c}
\end{align*}
$$

So we can progress in our derivatives:

$$
\nabla_c(-2{x}^\text{T}{Dc} + {c}^\text{T}{c})=0\\\\
-2{x}^\text{T}{D} + 2{c}=0\\\\
-2{D}^\text{T}{x} + 2{c}=0\\\\
{c}={D}^\text{T}{x}
$$

Great! We found the encoding function! Here are its dimensions:

<img src="https://github.com/rahiakela/mathematics-for-machine-learning/blob/main/deep-learning-book-maths/images/principal-components-analysis-PCA-encoding-function.png?raw=1" width="250" alt="Expression of the encoding function" title="The encoding function">
<em>The encoding function</em>

To go back from ${c}$ to ${x}$ we use $g({c})={Dc}$:

$$
r({x}) = g(f({x})={D}{D}^\text{T}{x}
$$

<img src="https://github.com/rahiakela/mathematics-for-machine-learning/blob/main/deep-learning-book-maths/images/principal-components-analysis-PCA-reconstruction-function.png?raw=1" width="300" alt="Expression of the reconstruction function" title="The reconstruction function">
<em>The reconstruction function</em>

## References

**PCA**

- A lot of intuitive explanations on PCA: https://arxiv.org/pdf/1404.1100.pdf

- https://brilliant.org/wiki/principal-component-analysis/#from-approximate-equality-to-minimizing-function

- http://www4.ncsu.edu/~slrace/LinearAlgebra2017/Slides/PCAPrint.pdf

- https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c

- https://www.cs.bgu.ac.il/~inabd171/wiki.files/lecture14_handouts.pdf

**Semi-orthogonal matrix**

- https://en.wikipedia.org/wiki/Semi-orthogonal_matrix

**Intuition about PCA**

- https://georgemdallas.wordpress.com/2013/10/30/principal-component-analysis-4-dummies-eigenvectors-eigenvalues-and-dimension-reduction/

**Derivatives**

- https://math.stackexchange.com/questions/1377764/derivative-of-vector-and-vector-transpose-product

**Link between variance maximized and error minimized**:

- https://stats.stackexchange.com/questions/130721/what-norm-of-the-reconstruction-error-is-minimized-by-the-low-rank-approximation

- https://stats.stackexchange.com/questions/32174/pca-objective-function-what-is-the-connection-between-maximizing-variance-and-m

- https://stats.stackexchange.com/questions/318625/why-do-the-leading-eigenvectors-of-a-maximize-texttrdtad

**Centering data**

- https://www.quora.com/Why-do-we-need-to-center-the-data-for-Principle-Components-Analysis
- https://stats.stackexchange.com/questions/22329/how-does-centering-the-data-get-rid-of-the-intercept-in-regression-and-pca

**Unit norm constraint**

- https://stats.stackexchange.com/questions/117695/why-is-the-eigenvector-in-pca-taken-to-be-unit-norm